In [180]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
from selenium import webdriver

# 1차 시도

일단 무작정 import하고 쓰기 시작했는데 chrome devtools에서는 데이터가 모두 보이는데 이렇게만 읽어오면 내가 원하는 데이터는 보이지 않는다.

In [111]:
res = requests.post('https://kbl.or.kr/game/archive-team')
print(res.content)

b'<!doctype html><html lang="ko"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no"/><title>KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80</title><meta name="author" content="KBL"/><meta name="description" content="KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80"/><meta name="referrer" content="no-referrer"/><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Expires" content="-1"/><meta http-equiv="Pragma" content="no-cache"><meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate"/><link rel="icon" href="/imgs/favicon/kbl/favicon.svg"><link rel="apple-touch-icon" type="image/png" id="apple-icon-60x60" sizes="60x60" href="/imgs/favicon/kbl/apple-icon-60x60.png"><link rel="apple-touch-icon" type="image/png" id="apple-icon-87x87" sizes="87x87" href="/imgs/favicon/kbl/apple-icon-87x87.png"><link r

In [92]:
soup = BeautifulSoup(res.content, 'html.parser')

In [97]:
soup.find_all('div', class_='archive-team-table01-wrap')

[]

# 2차 시도

검색해 본 결과, 위의 코드는 정적 페이지의 데이터를 읽어오는 것이고 동적 페이지는 webdriver를 사용해서 진행해야 내가 원하는 데이터를 가져올 수 있는 것이다.

- 그런데 아무리 사용해도 나오지 계속 같은 결과만을 가져왔다.
- 그러나~!!, ```driver.page_source```  하니까 나왔다 드디어!!!ㅜㅜ 감격스러움

- class를 더 쪼개고 해서 더 분할해서 데이터를 저장하는 것이 좋을 것 같다~

여기서 driver를 읽어오는 부분에거 경로를 정확하게 적어야 함 -> 아예 .exe로 하면 안된다.

In [199]:
driver = webdriver.Chrome('chromedriver_win32/')
# driver.implicitly_wait(3) # 있으나 없으나
driver.get('https://kbl.or.kr/game/archive-team')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_436728\2641466854.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


이거는 아님 ↓

In [182]:
from selenium.webdriver.common.by import By

In [183]:
driver.find_elements(By.CLASS_NAME, 'con-box')

[<selenium.webdriver.remote.webelement.WebElement (session="be5b8f01b60bc90fe6dc67a54977c15f", element="62ab578c-3752-420b-aeab-22f89cb7e30d")>]

이거는 아님 ↑

In [ ]:
html = driver.page_source # page_source를 가져와야 하는 것이었다ㅠㅠ 감격스러움 진짜
soup = BeautifulSoup(html, 'html.parser')

표 헤더 가져오기

In [213]:
header = soup.select('th') # 테이블 헤더 제목

for h in header:
    print(h.text.strip())

순위
팀
G
PTS
2P
2PA
2P%
3P
3PA
3P%
FG
FGA
FG%
FT
FTA
FT%
PP
PPA
PP%
REBOUND
AST
TO
STL
BS
PF
OFF
DEF
TOT


이건 필요 없을 듯

In [214]:
teams = soup.select('td.name') # 순위 순서로 팀 이름

for no in teams:
    print(no.text.strip())

서울 SK
울산 현대모비스
고양 캐롯
안양 KGC
전주 KCC
원주 DB
창원 LG
대구 한국가스공사
수원 KT
서울 삼성


header 밑에 데이터들 가져옴
- 맨 마지막 줄이 합계

In [219]:
rows = soup.select('tbody tr')

for r in rows:
    print(r.text.strip())

1서울 SK26
2울산 현대모비스25
2고양 캐롯25
4안양 KGC25
5전주 KCC25
6원주 DB25
7창원 LG25
8대구 한국가스공사25
9수원 KT25
10서울 삼성26
합계252
84.225.648.752.66.719.833.532.368.547.113.018.570.621.537.357.610.324.434.717.99.07.32.618.7
83.225.047.452.86.921.232.732.068.646.612.418.068.519.934.158.311.625.837.420.210.46.82.019.7
83.217.733.752.512.233.336.829.967.044.711.114.676.113.222.658.18.423.131.416.59.57.72.517.8
82.921.239.953.29.627.734.730.867.645.611.716.073.017.129.058.810.126.136.218.210.46.22.918.8
82.022.844.950.77.821.336.830.666.246.213.016.479.217.631.156.810.824.535.218.59.95.61.617.3
80.625.049.051.16.620.132.831.669.145.810.814.972.119.133.457.211.024.235.216.911.36.72.219.4
79.924.046.251.96.820.333.330.766.446.211.717.168.218.833.356.511.325.837.117.011.47.92.920.0
79.723.442.854.67.021.432.530.364.247.212.116.872.117.429.858.49.223.933.216.711.06.62.519.7
77.519.440.647.78.826.533.028.167.141.912.516.974.015.530.251.511.523.134.616.010.26.02.318.8
74.120.842.748.76.720.832.027.563.543.212.517.073.61

In [204]:
games = soup.select('td.bd-gray') # 각 팀 별 경기 횟수 마지막은 전체 경기 수

In [205]:
for game in games:
    print(game.text.strip())

26
25
25
25
25
25
25
25
25
26
252


In [220]:
scores = soup.select('td.on')

for score in scores:
    print(score.text.strip())

84.2
83.2
83.2
82.9
82.0
80.6
79.9
79.7
77.5
74.1


In [223]:
tdata = soup.select('tbody tr td') # 완전 요소 따로따로 가져올 때

for data in tdata:
    print(data.text.strip())

1
서울 SK
26
2
울산 현대모비스
25
2
고양 캐롯
25
4
안양 KGC
25
5
전주 KCC
25
6
원주 DB
25
7
창원 LG
25
8
대구 한국가스공사
25
9
수원 KT
25
10
서울 삼성
26
합계
252
84.2
25.6
48.7
52.6
6.7
19.8
33.5
32.3
68.5
47.1
13.0
18.5
70.6
21.5
37.3
57.6
10.3
24.4
34.7
17.9
9.0
7.3
2.6
18.7
83.2
25.0
47.4
52.8
6.9
21.2
32.7
32.0
68.6
46.6
12.4
18.0
68.5
19.9
34.1
58.3
11.6
25.8
37.4
20.2
10.4
6.8
2.0
19.7
83.2
17.7
33.7
52.5
12.2
33.3
36.8
29.9
67.0
44.7
11.1
14.6
76.1
13.2
22.6
58.1
8.4
23.1
31.4
16.5
9.5
7.7
2.5
17.8
82.9
21.2
39.9
53.2
9.6
27.7
34.7
30.8
67.6
45.6
11.7
16.0
73.0
17.1
29.0
58.8
10.1
26.1
36.2
18.2
10.4
6.2
2.9
18.8
82.0
22.8
44.9
50.7
7.8
21.3
36.8
30.6
66.2
46.2
13.0
16.4
79.2
17.6
31.1
56.8
10.8
24.5
35.2
18.5
9.9
5.6
1.6
17.3
80.6
25.0
49.0
51.1
6.6
20.1
32.8
31.6
69.1
45.8
10.8
14.9
72.1
19.1
33.4
57.2
11.0
24.2
35.2
16.9
11.3
6.7
2.2
19.4
79.9
24.0
46.2
51.9
6.8
20.3
33.3
30.7
66.4
46.2
11.7
17.1
68.2
18.8
33.3
56.5
11.3
25.8
37.1
17.0
11.4
7.9
2.9
20.0
79.7
23.4
42.8
54.6
7.0
21.4
32.5
30.3
64.2
47.2
12.1
16.

선수 기록실

In [165]:
driver = webdriver.Chrome('chromedriver_win32/')
driver.get('https://kbl.or.kr/game/archive-player')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_436728\389803932.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


In [196]:
records = soup.select('td')

for r in records:
    print(r.text.strip())

1
서울 SK
26
2
울산 현대모비스
25
2
고양 캐롯
25
4
안양 KGC
25
5
전주 KCC
25
6
원주 DB
25
7
창원 LG
25
8
대구 한국가스공사
25
9
수원 KT
25
10
서울 삼성
26
합계
252
84.2
25.6
48.7
52.6
6.7
19.8
33.5
32.3
68.5
47.1
13.0
18.5
70.6
21.5
37.3
57.6
10.3
24.4
34.7
17.9
9.0
7.3
2.6
18.7
83.2
25.0
47.4
52.8
6.9
21.2
32.7
32.0
68.6
46.6
12.4
18.0
68.5
19.9
34.1
58.3
11.6
25.8
37.4
20.2
10.4
6.8
2.0
19.7
83.2
17.7
33.7
52.5
12.2
33.3
36.8
29.9
67.0
44.7
11.1
14.6
76.1
13.2
22.6
58.1
8.4
23.1
31.4
16.5
9.5
7.7
2.5
17.8
82.9
21.2
39.9
53.2
9.6
27.7
34.7
30.8
67.6
45.6
11.7
16.0
73.0
17.1
29.0
58.8
10.1
26.1
36.2
18.2
10.4
6.2
2.9
18.8
82.0
22.8
44.9
50.7
7.8
21.3
36.8
30.6
66.2
46.2
13.0
16.4
79.2
17.6
31.1
56.8
10.8
24.5
35.2
18.5
9.9
5.6
1.6
17.3
80.6
25.0
49.0
51.1
6.6
20.1
32.8
31.6
69.1
45.8
10.8
14.9
72.1
19.1
33.4
57.2
11.0
24.2
35.2
16.9
11.3
6.7
2.2
19.4
79.9
24.0
46.2
51.9
6.8
20.3
33.3
30.7
66.4
46.2
11.7
17.1
68.2
18.8
33.3
56.5
11.3
25.8
37.1
17.0
11.4
7.9
2.9
20.0
79.7
23.4
42.8
54.6
7.0
21.4
32.5
30.3
64.2
47.2
12.1
16.

In [198]:
records = soup.select('div.top-scroll-table.archive-player-table01 td')

for r in records:
    print(r.text.strip())

- 기록실 링크 정리

```https://kbl.or/game/*```

1. 일정: schedule-list
2. 팀 순위: team-rank
3. 선수 순위: player-rank-top5
4. 팀 기록실: archive-team
5. 선수 기록실: archive-player
6. 팀 기록 비교: compare-team
7. 공수력 비교: compare-offence-defence